In [ ]:
pip install rdflib owlready2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 67.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.0/569.0 kB 34.5 MB/s eta 0:00:00


In [ ]:
pip install  graphviz pydotplus -q

In [ ]:
!apt-get install -y openjdk-11-jdk   -qq

Selecting previously unselected package libatspi2.0-0:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../00-libatspi2.0-0_2.44.0-3_amd64.deb ...
Unpacking libatspi2.0-0:amd64 (2.44.0-3) ...
Selecting previously unselected package libxtst6:amd64.
Preparing to unpack .../01-libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package session-migration.
Preparing to unpack .../02-session-migration_0.3.6_amd64.deb ...
Unpacking session-migration (0.3.6) ...
Selecting previously unselected package gsettings-desktop-schemas.
Preparing to unpack .../03-gsettings-desktop-schemas_42.0-1ubuntu1_all.deb ...
Unpacking gsettings-desktop-schemas (42.0-1ubuntu1) ...
Selecting previously unselected package at-spi2-core.
Preparing to unpack .../04-at-spi2-core_2.44.0-3_amd64.deb ...
Unpacking at-spi2-core (2.44.0-3) ...
Selecting previously unselected package fonts-dejavu-core.
Prepar

In [ ]:
import pandas as pd
import os
from owlready2 import *

## Загрузка данных

In [ ]:
# Загружаем данные из CSV файлов
def load_data():
    knowledge_df = pd.read_csv('ontology_data/knowledge.csv')
    competencies_df = pd.read_csv('ontology_data/competencies.csv')
    competency_knowledge_df = pd.read_csv('ontology_data/competency_knowledge.csv')
    positions_df = pd.read_csv('ontology_data/positions.csv')
    position_competency_df = pd.read_csv('ontology_data/position_competency.csv')
    employees_df = pd.read_csv('ontology_data/employees.csv')

    return knowledge_df, competencies_df, competency_knowledge_df, positions_df, position_competency_df, employees_df

In [ ]:
knowledge_df, competencies_df, competency_knowledge_df, positions_df, position_competency_df, employees_df = load_data()

In [ ]:
knowledge_df

,knowledge_id,knowledge_name
0,1,Математическая статистика
1,2,Линейная алгебра
2,3,Теория вероятностей
3,4,Python программирование
4,5,"Библиотеки ML (scikit-learn, TensorFlow, PyTorch)"
5,6,Обработка естественного языка (NLP)
6,7,Компьютерное зрение
7,8,"Работа с большими данными (Spark, Hadoop)"
8,9,SQL и базы данных
9,10,Визуализация данных


# Создание новой онтологии

## Итерация 1.
Создадим знания и компетенции

In [ ]:
# Создаем онтологию
onto = get_ontology("http://test.org/ml_specialist_v1.owl")

with onto:
    class Знание(Thing):
        pass

    class Компетенция(Thing):
        pass

    class требует_знание(ObjectProperty):
        domain = [Компетенция]
        range = [Знание]

    class название(DataProperty):
        domain = [Thing]
        range = [str]

### Создание индивидуальностей для знаний и компетенций



In [ ]:
def create_knowledge(knowledge_df):
  with onto:

    knowledge_instances = {}
    for _, row in knowledge_df.iterrows():
        knowledge = Знание(f"знание_{row['knowledge_id']}")
        knowledge.название = [row['knowledge_name']]
        knowledge_instances[row['knowledge_id']] = knowledge
    return knowledge_instances

def create_competencies( competencies_df):
  with onto:
    competency_instances = {}
    for _, row in competencies_df.iterrows():
        competency = Компетенция(f"компетенция_{row['competency_id']}")
        competency.название = [row['competency_name']]
        competency_instances[row['competency_id']] = competency
    return competency_instances


In [ ]:
knowledges = create_knowledge(knowledge_df)

In [ ]:
knowledges

{1: ml_specialist_v1.знание_1,
 2: ml_specialist_v1.знание_2,
 3: ml_specialist_v1.знание_3,
 4: ml_specialist_v1.знание_4,
 5: ml_specialist_v1.знание_5,
 6: ml_specialist_v1.знание_6,
 7: ml_specialist_v1.знание_7,
 8: ml_specialist_v1.знание_8,
 9: ml_specialist_v1.знание_9,
 10: ml_specialist_v1.знание_10}

In [ ]:
for id, item in knowledges.items():
  print (item.название[0])

Математическая статистика
Линейная алгебра
Теория вероятностей
Python программирование
Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
Обработка естественного языка (NLP)
Компьютерное зрение
Работа с большими данными (Spark, Hadoop)
SQL и базы данных
Визуализация данных


In [ ]:
competencies = create_competencies( competencies_df)
for id, item in competencies.items():
  print (item.название[0])

Математическая подготовка
Программирование и алгоритмы
Машинное обучение
Глубокое обучение
Обработка и анализ данных
Работа с большими данными
Визуализация и представление данных
Развертывание ML-моделей


### Создание отношений между знаниями и компетенциями

In [ ]:
# Связываем компетенции со знаниями
def  relate_competencies_knowledge (competency_knowledge_df,  competency_instances, knowledge_instances):
  with onto:
    for _, row in competency_knowledge_df.iterrows():
        competency = competency_instances[row['competency_id']]
        knowledge = knowledge_instances[row['knowledge_id']]
        competency.требует_знание.append(knowledge)


In [ ]:
relate_competencies_knowledge (competency_knowledge_df, competencies, knowledges)

Выведем связи между компенциями и знаниями

In [ ]:
def print_competency_knowledge():
  print("\n" + "="*60)
  print("СВЯЗИ ЗНАНИЙ И КОМПЕТЕНЦИЙ")
  print("="*60)

  for competency_id, competency in competencies.items():
      comp_name = competency.название[0]
      print(f"\n{comp_name} требует знания:")

      required_knowledge = list(competency.требует_знание)
      for know in required_knowledge:
          print(f"  - {know.название[0]}")

print_competency_knowledge()



СВЯЗИ ЗНАНИЙ И КОМПЕТЕНЦИЙ

Математическая подготовка требует знания:
  - Математическая статистика
  - Линейная алгебра
  - Теория вероятностей

Программирование и алгоритмы требует знания:
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Машинное обучение требует знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Глубокое обучение требует знания:
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
  - Обработка естественного языка (NLP)
  - Компьютерное зрение

Обработка и анализ данных требует знания:
  - Математическая статистика
  - SQL и базы данных

Работа с большими данными требует знания:
  - Работа с большими данными (Spark, Hadoop)
  - SQL и базы данных

Визуализация и представление данных требует знания:
  - Визуализация данных
  - Python программирование

Развертывание ML-моделей требует знания:
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, 

# Итерация 2

Добавление новых концептов в онтологию


In [ ]:
with onto:
  class Специалист(Thing):
        pass

  class Должность(Thing):
        pass

  class обладает_знанием(ObjectProperty):
        domain = [Специалист]
        range = [Знание]

  class обладает_компетенцией(ObjectProperty):
        domain = [Специалист]
        range = [Компетенция]

  class может_занимать_должность(ObjectProperty):
        domain = [Специалист]
        range = [Должность]

  class требует_компетенцию(ObjectProperty):
        domain = [Должность]
        range = [Компетенция]

  class имя_сотрудника(DataProperty):
        domain = [Специалист]
        range = [str]

In [ ]:
employees_df.groupby(['employee_id', 'full_name'])['knowledge_id'].apply(list)

,,knowledge_id
employee_id,full_name,
1,Иванов Алексей Сергеевич,"[1, 4, 5]"
2,Петрова Мария Владимировна,"[1, 4, 5]"
3,Сидоров Дмитрий Петрович,"[1, 2, 9]"
4,Козлова Анна Игоревна,"[5, 6, 7]"
5,Федоров Максим Андреевич,"[5, 6, 7]"
6,Николаева Екатерина Дмитриевна,"[4, 5, 10]"
7,Орлов Сергей Викторович,"[1, 2, 3]"


### Создание индивидуальностей для сотрудников и их отношений с знаниями


In [ ]:
def create_employees(employees_df, knowledge_instances):
  employees_df = employees_df.drop_duplicates()
  employee_knowledge = employees_df.groupby(['employee_id', 'full_name'])['knowledge_id'] \
  .apply(lambda x: list(set(x))).reset_index()

  employee_instances = {}
  for _, row in employee_knowledge.iterrows():
    # создадим сотрудника
      employee = Специалист(f"сотрудник_{row['employee_id']}")
      employee.имя_сотрудника = [row['full_name']]
      employee_instances[row['employee_id']] = employee

      # добавим знания сотруднику
      for knowledge_id in row['knowledge_id']:
          if knowledge_id in knowledge_instances:
              employee.обладает_знанием.append(knowledge_instances[knowledge_id])

  return employee_instances

In [ ]:
employees = create_employees(employees_df, knowledges)

In [ ]:
def print_employee_mindmap():
  print("\n" + "="*60)
  print("КАРТА ЗНАНИЙ КАЖДОГО СОТРУДНИКА")
  print("="*60)
  for employee_id, employee in employees.items():
          employee_name = employee.имя_сотрудника[0] if employee.имя_сотрудника else f"Сотрудник {employee_id}"

          print(f"\n{employee_name}:")

          # Получаем список знаний сотрудника
          knowledge_list = list(employee.обладает_знанием)

          if knowledge_list:
              print("Знания:")
              for knowledge in knowledge_list:
                  know_name = knowledge.название[0] if knowledge.название else knowledge.name
                  print(f"  - {know_name}")
          else:
              print("Знания: нет")

print_employee_mindmap()


КАРТА ЗНАНИЙ КАЖДОГО СОТРУДНИКА

Иванов Алексей Сергеевич:
Знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Петрова Мария Владимировна:
Знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Сидоров Дмитрий Петрович:
Знания:
  - Математическая статистика
  - Линейная алгебра
  - SQL и базы данных

Козлова Анна Игоревна:
Знания:
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
  - Обработка естественного языка (NLP)
  - Компьютерное зрение

Федоров Максим Андреевич:
Знания:
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
  - Обработка естественного языка (NLP)
  - Компьютерное зрение

Николаева Екатерина Дмитриевна:
Знания:
  - Визуализация данных
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Орлов Сергей Викторович:
Знания:
  - Математическая статистика
  - Линейная алгебра
  - Теория вероятностей


### Создание индивидуальностей для должностей и из связей с компетенциями


In [ ]:
# Создаем экземпляры должностей
def create_position_instances(positions_df):
    position_instances = {}
    for _, row in positions_df.iterrows():
        position = Должность(f"должность_{row['position_id']}")
        position.name = row['position_name']
        position_instances[row['position_id']] = position
    return position_instances

In [ ]:
positions = create_position_instances(positions_df)
positions

{1: ml_specialist_v1.ML Engineer,
 2: ml_specialist_v1.Data Scientist,
 3: ml_specialist_v1.Data Analyst,
 4: ml_specialist_v1.NLP Engineer,
 5: ml_specialist_v1.Computer Vision Engineer}

In [ ]:
# Устанавливаем связи между должностями и компетенциями
def set_position_competency_relations(position_competency_df, position_instances, competency_instances):
    for _, row in position_competency_df.iterrows():
        position = position_instances[row['position_id']]
        competency = competency_instances[row['competency_id']]
        position.требует_компетенцию.append(competency)

In [ ]:
set_position_competency_relations(position_competency_df, positions, competencies)

In [ ]:
# сюда вставить код для выполнения самостоятельных заданий
positions[4].требует_компетенцию.append(competencies[4])



## Выведем требования к должностям

In [ ]:
# Проверим требования должностей
def print_position_require():
  print("\n" + "="*60)
  print("ТРЕБОВАНИЯ ДОЛЖНОСТЕЙ")
  print("="*60)

  for position_id, position in positions.items():
      pos_name = position.name
      print(f"\n{pos_name} требует компетенции:")

      required_competencies = list(position.требует_компетенцию)
      if required_competencies:
          for comp in required_competencies:
              print(f"  - {comp.название[0]}")
      else:
          print("  - нет требований")

print_position_require()


ТРЕБОВАНИЯ ДОЛЖНОСТЕЙ

ML Engineer требует компетенции:
  - Программирование и алгоритмы
  - Машинное обучение

Data Scientist требует компетенции:
  - Программирование и алгоритмы
  - Машинное обучение

Data Analyst требует компетенции:
  - Обработка и анализ данных

NLP Engineer требует компетенции:
  - Программирование и алгоритмы
  - Глубокое обучение

Computer Vision Engineer требует компетенции:
  - Машинное обучение


## Создание аксиом для онтологии

 <b>Аксиома 1:</b> Если специалист обладает всеми требуемыми знаниями для компетенции, то у него есть эта компетенция

 <b>Аксиома 2:</b> Если у специалиста есть требуемые компетенции для должности, то он может занимать эту должность

  <b> Аксиома 3:</b> (для проверки корректности онтологии) Если специалист занимает должность, то у него должны быть соответствующие компетенции
      


In [ ]:
def define_rules():
    # Аксиома 1:
    with onto:
        class ИмеетКомпетенциюПоЗнаниям(Специалист >> bool):
            def __init__(self):
                super().__init__()

            def __call__(self, specialist):
                # Для каждой компетенции проверяем, есть ли у специалиста все необходимые знания
                for competency in onto.Компетенция.instances():
                    required_knowledge = list(competency.требует_знание)
                    specialist_knowledge = list(specialist.обладает_знанием)

                    # Проверяем, что все требуемые знания есть у специалиста
                    if all(knowledge in specialist_knowledge for knowledge in required_knowledge):
                        if competency not in specialist.обладает_компетенцией:
                            specialist.обладает_компетенцией.append(competency)

                return True

        # Аксиома 2:
        class МожетЗаниматьДолжностьПоКомпетенциям(Специалист >> bool):
            def __init__(self):
                super().__init__()

            def __call__(self, specialist):
                # Для каждой должности проверяем, есть ли у специалиста все необходимые компетенции
                for position in onto.Должность.instances():
                    required_competencies = list(position.требует_компетенцию)
                    specialist_competencies = list(specialist.обладает_компетенцией)

                    # Проверяем, что все требуемые компетенции есть у специалиста
                    if all(competency in specialist_competencies for competency in required_competencies):
                        if position not in specialist.может_занимать_должность:
                            specialist.может_занимать_должность.append(position)

                return True

        # Аксиома 3:
        class ПроверитьКомпетенцииДляДолжности(Специалист >> bool):
            def __init__(self):
                super().__init__()

            def __call__(self, specialist):
                positions = list(specialist.может_занимать_должность)
                competencies = list(specialist.обладает_компетенцией)

                for position in positions:
                    required_competencies = list(position.требует_компетенцию)
                    if not all(comp in competencies for comp in required_competencies):
                        print(f"Предупреждение: у {specialist.name} нет всех компетенций для {position.name}")

                return True

    return ИмеетКомпетенциюПоЗнаниям(), МожетЗаниматьДолжностьПоКомпетенциям(), ПроверитьКомпетенцииДляДолжности()

In [ ]:
rule1, rule2, rule3 = define_rules()

In [ ]:
# Проверим связи знаний и компетенций
print_competency_knowledge()


СВЯЗИ ЗНАНИЙ И КОМПЕТЕНЦИЙ

Математическая подготовка требует знания:
  - Математическая статистика
  - Линейная алгебра
  - Теория вероятностей

Программирование и алгоритмы требует знания:
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Машинное обучение требует знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)

Глубокое обучение требует знания:
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
  - Обработка естественного языка (NLP)
  - Компьютерное зрение

Обработка и анализ данных требует знания:
  - Математическая статистика
  - SQL и базы данных

Работа с большими данными требует знания:
  - Работа с большими данными (Spark, Hadoop)
  - SQL и базы данных

Визуализация и представление данных требует знания:
  - Визуализация данных
  - Python программирование

Развертывание ML-моделей требует знания:
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, 

In [ ]:
onto.Специалист.instances()

[ml_specialist_v1.сотрудник_1,
 ml_specialist_v1.сотрудник_2,
 ml_specialist_v1.сотрудник_3,
 ml_specialist_v1.сотрудник_4,
 ml_specialist_v1.сотрудник_5,
 ml_specialist_v1.сотрудник_6,
 ml_specialist_v1.сотрудник_7]

In [ ]:
for employee in onto.Специалист.instances():
        rule1(employee)  # Применяем правило для компетенций
        rule2(employee)  # Применяем правило для должностей
        rule3(employee)  # Проверяем соответствие

## Проведение анализа данных

In [ ]:
def print_position_for_employee():
  print("\n" + "="*60)
  print("АНАЛИЗ ВОЗМОЖНЫХ ДОЛЖНОСТЕЙ У СОТРУДНИКОВ")
  print("="*60)

  for employee in onto.Специалист.instances():
          print(f"\nСотрудник: {employee.имя_сотрудника[0]}")

          print("Знания:")
          for knowledge in employee.обладает_знанием:
              print(f"  - {knowledge.название[0]}")

          print("Компетенции:")
          for competency in employee.обладает_компетенцией:
              print(f"  - {competency.название[0]}")

          print("Может занимать должности:")
          for position in employee.может_занимать_должность:
              print(f"  - {position.name}")

          print("-" * 30)

print_position_for_employee()


АНАЛИЗ ВОЗМОЖНЫХ ДОЛЖНОСТЕЙ У СОТРУДНИКОВ

Сотрудник: Иванов Алексей Сергеевич
Знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
Компетенции:
  - Программирование и алгоритмы
  - Машинное обучение
  - Развертывание ML-моделей
Может занимать должности:
  - ML Engineer
  - Data Scientist
  - Computer Vision Engineer
------------------------------

Сотрудник: Петрова Мария Владимировна
Знания:
  - Математическая статистика
  - Python программирование
  - Библиотеки ML (scikit-learn, TensorFlow, PyTorch)
Компетенции:
  - Программирование и алгоритмы
  - Машинное обучение
  - Развертывание ML-моделей
Может занимать должности:
  - ML Engineer
  - Data Scientist
  - Computer Vision Engineer
------------------------------

Сотрудник: Сидоров Дмитрий Петрович
Знания:
  - Математическая статистика
  - Линейная алгебра
  - SQL и базы данных
Компетенции:
  - Обработка и анализ данных
Может занимать должности:
  - Data Analyst
-

In [ ]:
# Убедимся, что у сотрудников достаточно знаний для получения компетенций
def check_employee_competencies(employee_instances, competency_instances):
    """Проверяем, какие компетенции могут получить сотрудники"""
    print("\n" + "="*60)
    print("ПРОВЕРКА ВОЗМОЖНЫХ КОМПЕТЕНЦИЙ СОТРУДНИКОВ")
    print("="*60)

    for employee_id, employee in employee_instances.items():
        employee_name = employee.имя_сотрудника[0]
        print(f"\n{employee_name}:")

        for competency_id, competency in competency_instances.items():
            comp_name = competency.название[0]
            required_knowledge = set(competency.требует_знание)
            employee_knowledge = set(employee.обладает_знанием)

            # Проверяем, есть ли у сотрудника все необходимые знания
            if required_knowledge.issubset(employee_knowledge):
                print(f"  Может получить компетенцию: {comp_name}")
                # Присваиваем компетенцию
                if competency not in employee.обладает_компетенцией:
                    employee.обладает_компетенцией.append(competency)
            else:
                missing_knowledge = required_knowledge - employee_knowledge
                if missing_knowledge:
                    print(f"  Не хватает для '{comp_name}':")
                    for know in missing_knowledge:
                        print(f"      - {know.название[0]}")


In [ ]:
check_employee_competencies(employees, competencies)


ПРОВЕРКА ВОЗМОЖНЫХ КОМПЕТЕНЦИЙ СОТРУДНИКОВ

Иванов Алексей Сергеевич:
  Не хватает для 'Математическая подготовка':
      - Теория вероятностей
      - Линейная алгебра
  Может получить компетенцию: Программирование и алгоритмы
  Может получить компетенцию: Машинное обучение
  Не хватает для 'Глубокое обучение':
      - Компьютерное зрение
      - Обработка естественного языка (NLP)
  Не хватает для 'Обработка и анализ данных':
      - SQL и базы данных
  Не хватает для 'Работа с большими данными':
      - Работа с большими данными (Spark, Hadoop)
      - SQL и базы данных
  Не хватает для 'Визуализация и представление данных':
      - Визуализация данных
  Может получить компетенцию: Развертывание ML-моделей

Петрова Мария Владимировна:
  Не хватает для 'Математическая подготовка':
      - Теория вероятностей
      - Линейная алгебра
  Может получить компетенцию: Программирование и алгоритмы
  Может получить компетенцию: Машинное обучение
  Не хватает для 'Глубокое обучение':
      -

In [ ]:
def check_employee_positions(employee_instances, position_instances):
    """Проверяем, какие должности могут занять сотрудники"""
    print("\n" + "="*60)
    print("ПРОВЕРКА ВОЗМОЖНЫХ ДОЛЖНОСТЕЙ СОТРУДНИКОВ")
    print("="*60)

    for employee_id, employee in employee_instances.items():
        employee_name = employee.имя_сотрудника[0]
        print(f"\n{employee_name}:")

        for position_id, position in position_instances.items():
            pos_name = position.name
            required_competencies = set(position.требует_компетенцию)
            employee_competencies = set(employee.обладает_компетенцией)

            # Проверяем, есть ли у сотрудника все необходимые компетенции
            if required_competencies.issubset(employee_competencies):
                print(f"   Может занять должность: {pos_name}")
                # Присваиваем должность
                if position not in employee.может_занимать_должность:
                    employee.может_занимать_должность.append(position)
            else:
                missing_competencies = required_competencies - employee_competencies
                if missing_competencies:
                    print(f"   Не хватает для '{pos_name}':")
                    for comp in missing_competencies:
                        print(f"      - {comp.название[0]}")

# Применяем проверки

check_employee_positions(employees, positions)



ПРОВЕРКА ВОЗМОЖНЫХ ДОЛЖНОСТЕЙ СОТРУДНИКОВ

Иванов Алексей Сергеевич:
   Может занять должность: ML Engineer
   Может занять должность: Data Scientist
   Не хватает для 'Data Analyst':
      - Обработка и анализ данных
   Не хватает для 'NLP Engineer':
      - Глубокое обучение
   Может занять должность: Computer Vision Engineer

Петрова Мария Владимировна:
   Может занять должность: ML Engineer
   Может занять должность: Data Scientist
   Не хватает для 'Data Analyst':
      - Обработка и анализ данных
   Не хватает для 'NLP Engineer':
      - Глубокое обучение
   Может занять должность: Computer Vision Engineer

Сидоров Дмитрий Петрович:
   Не хватает для 'ML Engineer':
      - Машинное обучение
      - Программирование и алгоритмы
   Не хватает для 'Data Scientist':
      - Машинное обучение
      - Программирование и алгоритмы
   Может занять должность: Data Analyst
   Не хватает для 'NLP Engineer':
      - Глубокое обучение
      - Программирование и алгоритмы
   Не хватает для 'C

# Задание 1:  Добавление необходимой компетенции в должность

Добавить требование, что компетениция NLP Enginner содержит знание по глубокому обучению.

<B>Важно, что добавление в онтологию новых данных в Питоне должно сопровождаться созданием новой онтологии.</B>


In [ ]:
positions

{1: ml_specialist_v1.ML Engineer,
 2: ml_specialist_v1.Data Scientist,
 3: ml_specialist_v1.Data Analyst,
 4: ml_specialist_v1.NLP Engineer,
 5: ml_specialist_v1.Computer Vision Engineer}

In [ ]:
positions[4].требует_компетенцию

[ml_specialist_v1.компетенция_2, ml_specialist_v1.компетенция_4]

In [ ]:
competencies[2].название

['Программирование и алгоритмы']

In [ ]:
for i, item in competencies.items():
  print (i, ' ', item.название)


1   ['Математическая подготовка']
2   ['Программирование и алгоритмы']
3   ['Машинное обучение']
4   ['Глубокое обучение']
5   ['Обработка и анализ данных']
6   ['Работа с большими данными']
7   ['Визуализация и представление данных']
8   ['Развертывание ML-моделей']


In [ ]:
positions[4].требует_компетенцию.append(competencies[4])

###

## Задание 2:
Добавьте требование в онтологию: должность Data Scientist требует компетенции "Математическая подготовка"

## Задание 3:
Добавьте сотруднику Федорову Максиму Андреевичу необходимые знания, чтобы он мог претендовать на должность Computer Vision Engineer

## Задание 4:
Добавьте требование в онтологию нового сотрудника, укажите в качестве имя_сотрудника ваше ФИО. Добавьте знания для данного сотрудника, чтобы он смог претендовать на все должности текущей онтологии